In [44]:
import requests
import json
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon

In [15]:
CLIENT_ID = "TLIYr5sSlL7AGheLZ0ruA"
API_KEY = "UnQvi6WM4kG4wodZgTsR4GSdF8Bva1U3faS1dp0DzLcBAHujLyX2_5puEBrhtE4xXKxZpJG2iBb8s4ulucSjMTEgD-TmLz_wkLLKr38SqPiT0-15ouO35RjaB7jPXXYx"

headers = {"Authorization": "Bearer {}".format(API_KEY)}

In [23]:
url='https://api.yelp.com/v3/businesses/search'

In [25]:
restaurants = []
limit = 50

for offset in range(0, 1000, limit):
    req=requests.get(url, params=params, headers=headers)
    params = {'location': 'Chicago', "limit": limit, "offset": offset}
    if req.status_code == 200:
        restaurants += json.loads(req.text)["businesses"]
    else:
        print("Error")

In [54]:
yelp = pd.DataFrame(restaurants)
yelp.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & the Goat,https://s3-media1.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,8475,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 41.884176, 'longitude': -87.64794...",[],$$$,"{'address1': '809 W Randolph St', 'address2': ...",+13124926262,(312) 492-6262,3396.525043
1,cKZNbMvoqJaUe7n6lf6i7w,wildberry-pancakes-and-cafe-chicago-2,Wildberry Pancakes and Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/or0SLp...,False,https://www.yelp.com/biz/wildberry-pancakes-an...,7099,"[{'alias': 'pancakes', 'title': 'Pancakes'}, {...",4.5,"{'latitude': 41.884668, 'longitude': -87.62288}",[pickup],$$,"{'address1': '130 E Randolph St', 'address2': ...",+13129389777,(312) 938-9777,5082.221755
2,xoi7Cw7FoknAx5p880RtWQ,au-cheval-chicago,Au Cheval,https://s3-media4.fl.yelpcdn.com/bphoto/a0yhvS...,False,https://www.yelp.com/biz/au-cheval-chicago?adj...,6860,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 41.88466, 'longitude': -87.647668}",[],$$,"{'address1': '800 W Randolph St', 'address2': ...",+13129294580,(312) 929-4580,3376.598427
3,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media2.fl.yelpcdn.com/bphoto/Qf9YAh...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,7025,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.0,"{'latitude': 41.890694, 'longitude': -87.624782}",[],$$,"{'address1': '444 N Michigan Ave', 'address2':...",+13124641744,(312) 464-1744,4670.876907
4,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/cv0OrG...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,5976,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[delivery, pickup]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,4000.999053


In [47]:
inspections = pd.read_csv("EPFL/CS401/Food-Quality-and-Social-Groups-in-Chicago/data/food-inspections.csv")
columns_to_keep = ["Inspection ID", "DBA Name", "AKA Name", "Risk", "Results", "Latitude", "Longitude", "Violations", "Inspection Date"]
inspections = inspections[columns_to_keep]
inspections.columns = ["inspection_id", "dba_name", "aka_name", "risk", "results", "latitude", "longitude", "violations", "date"]
inspections.columns

def map_risk(x_):
    x = str(x_)
    if "High" in x:
        return 3
    elif "Medium" in x:
        return 2
    elif "Low" in x:
        return 1
    else:
        return x
    
def map_n_violations(x_):
    x = str(x_)
    return x.split(".")[0]
    
inspections["date"] = pd.to_datetime(inspections["date"])
inspections = inspections.dropna(subset=["latitude", "longitude"])
inspections = inspections[inspections["results"].isin(["Pass", "Pass w/ Conditions", "Fail"])]
inspections["risk"] = inspections["risk"].apply(map_risk)
inspections = inspections[inspections["risk"].isin([1, 2, 3])]
inspections["risk"] = inspections["risk"].astype("int16")
inspections["n_violations"] = inspections["violations"].fillna(0).apply(map_n_violations)
inspections["n_violations"] = inspections["n_violations"].astype("int32")

In [48]:
geo_df = gpd.read_file("EPFL/CS401/Food-Quality-and-Social-Groups-in-Chicago/data/chicago_boundaries_community_area.geojson")
geo_df = geo_df[["community", "shape_area", "shape_len", "geometry"]]

In [49]:
def get_nearest_neigh(point, geo_df):
    idx = geo_df.geometry.distance(point).idxmin()
    return geo_df.loc[idx, 'community']

## key: key that will be used to join
def assign_community(geo_df, data, latitude="latitude", longitude="longitude", verbose=False):
    geometry = [Point(x, y) for x, y in zip(data[longitude], data[latitude])]
    crs = {'init': 'epsg:4326'}
    data_to_join = gpd.GeoDataFrame(data, 
                                       crs=crs,
                                       geometry=geometry)
    points_to_neigh = sjoin(data_to_join, geo_df, how='left')
    
    neigh_not_found = points_to_neigh[pd.isna(points_to_neigh['community'])]
    
    if verbose:
        print("There are {} points without an exact neighborhood".format(len(neigh_not_found)))
    
    neigh_not_found['community'] = neigh_not_found.copy().geometry.apply(get_nearest_neigh, geo_df=geo_df)
    points_to_neigh.loc[neigh_not_found.index] = neigh_not_found
    
    if verbose:
        print("There are {} points without an exact neighborhood"\
              .format(len(points_to_neigh[pd.isna(points_to_neigh['community'])])))
    
    columns_to_return = list(data.columns) + ["community"]
    return points_to_neigh[columns_to_return]

In [53]:
inspections_community = assign_community(geo_df, inspections, verbose=True)

There are 2665 points without an exact neighborhood
There are 0 points without an exact neighborhood


C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [88]:
inspections_community[inspections_community["date"].dt.year == 2019].groupby("community")["dba_name"].nunique()

community
ALBANY PARK       139
ARCHER HEIGHTS     57
ARMOUR SQUARE      93
ASHBURN            93
AUBURN GRESHAM     77
                 ... 
WEST LAWN         102
WEST PULLMAN       26
WEST RIDGE        244
WEST TOWN         385
WOODLAWN           23
Name: dba_name, Length: 76, dtype: int64

In [89]:
tmp = pd.DataFrame([yelp["coordinates"].apply(lambda x: x["latitude"]),
                    yelp["coordinates"].apply(lambda x: x["longitude"])]).transpose()

tmp.columns = ["latitude", "longitude"]
yelp[["latitude", "longitude"]] = tmp

In [90]:
yelp_community = assign_community(geo_df, yelp, verbose=True)

There are 1 points without an exact neighborhood
There are 0 points without an exact neighborhood


C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:
yelp_community.groupby("community").count()["id"]

community
ALBANY PARK            10
ARCHER HEIGHTS          1
ARMOUR SQUARE          16
AVONDALE                9
BELMONT CRAGIN          3
BRIDGEPORT              6
DUNNING                 3
EAST GARFIELD PARK      2
EDGEWATER              31
FOREST GLEN             2
HUMBOLDT PARK           3
IRVING PARK             9
JEFFERSON PARK          1
KENWOOD                 1
LAKE VIEW             107
LINCOLN PARK           81
LINCOLN SQUARE         27
LOGAN SQUARE           67
LOOP                   78
LOWER WEST SIDE        22
NEAR NORTH SIDE       202
NEAR SOUTH SIDE        13
NEAR WEST SIDE         88
NORTH CENTER           16
NORTH PARK              3
NORWOOD PARK            4
PORTAGE PARK            6
ROGERS PARK             6
SOUTH LAWNDALE          1
UPTOWN                 26
WEST RIDGE              3
WEST TOWN             123
Name: id, dtype: int64